***
# Assignment 1 Social Graphs 2023

**Link to assignment description:** https://github.com/SocialComplexityLab/socialgraphs2023/blob/main/assignments/Assignment1.ipynb
***

## Table of Contents
* [0. Requirements](#requirements)
* [1. Motivation](#motivation)
    * [1.1 Dataset](#dateset)
    * [1.2 Dataset Description](#datasetdescription)
* [2. Basic Stats](#basicstats)
    * [2.1 Handling Missing Values](#missingvalues)
    * [2.2 Dataset Exploration](#datasetexploration)
    * [2.3 Zooming in on Air Quality](#zoomair)
        * [2.3.1 Specifying Target Variable](#targetvar)
        * [2.3.2 Air Quality using "Poor Air Quality"](#targetvar)
        * [2.3.3 Air Quality using "Hazardous Air Quality"](#targetvar)
    * [2.4 EDA Continued](#edaplots)
        * [2.4.1 Correlation Matrix](corrmatrix)
* [3. Data Analysis](#dataanalysis)
    * [3.1 Clustering based on Transportation Methods](#clustering)
        * [3.1.1 Optimal Number of Clusters](#numberofclusters)
        * [3.1.2 Analyzing the Properties of the Clusters](#clusterproperties)
    * [3.2 Linear Regression Model](#linreg)
        * [3.2.1 Unclustered Dataset Regression](#fulldata)
        * [3.2.2 Cluster 0 Regression](#cluster0)
        * [3.2.3 Cluster 1 Regression](#cluster1)
        * [3.2.4 Cluster 2 Regression](#cluster2)
* [4. Genre](#genre)
* [5. Final Visualizations](#finalviz)
    * [5.1 AQI Heatmap](#heatmap)
    * [5.2 US Heatmap](#heatmap)
    * [5.3 Stacked Barchart](#barchart)
* [6. Discussion](#discussion)
    * [6.1 Successes](#success)
    * [6.2 Limitations](#limitations)
    * [6.3 Improvements](#improvements)
* [7. Contributions](#contributions)
* [8. References](#references)

## 0. Requirements <a class="anchor" id="requirements"></a>

In [4]:
import networkx as nx
import numpy as np

## 1. Binning degree distributions <a class="anchor" id="motivation"></a>


### 1.1 Dataset <a class="anchor" id="dataset"></a>

### 1.2 Dataset Description <a class="anchor" id="datasetdescription"></a>


# 2. Random networks and plotting in `networkx` <a class="anchor" id="basicstats"></a>

We only keep the attributes of interest for our analysis, the rest of the columns are dropped. 

### 2.1. Handling Missing Values  <a class="anchor" id="missingvalues"></a>


### 2.2. Data Exploration <a class="anchor" id="datasetexploration"></a>

How many cities do we have?

### 2.3. Zooming in on air quality <a class="anchor" id="zoomair"></a>


### 2.4 EDA Continued <a class="anchor" id="edaplots"></a>

## 3. Data analysis <a class="anchor" id="dataanalysis"></a>
- Describe your data analysis and explain what you've learned about the dataset.
- If relevant, talk about your machine-learning.

**In the following**, we want to perform clustering across cities based on transportation methods. **We then** want to discover if the clusters we find hold any interesting patterns. **Finally**, we want to analyze whether transportation methods actually have a statistically significant effect on the Air Quality using a multivariate linear regression model. 

**Before we do so**, we must preprocess the data a tiny bit more. 

### 3.1 Clustering based on Transportation Methods <a class="anchor" id="clustering"></a>


#### 3.1.1 Optimal Number of Clusters <a class="anchor" id="numberofclusters"></a>

#### 3.1.2 Analyzing the Properties of the Clusters <a class="anchor" id="clusterproperties"></a>

### 3.2 Linear Regression <a class="anchor" id="linreg"></a>


#### 3.2.1 Unclustered Dataset Regression <a class="anchor" id="fulldata"></a>

## 4. Genre. Which genre of data story did you use? <a class="anchor" id="genre"></a>

## 5. Final visualizations <a class="anchor" id="finalviz"></a>

### 5.2 Stacked Barchart <a class="anchor" id="barchart"></a>

### 5.3 Means for each 3 clusters <a class="anchor" id="clustermeans"></a>

## 6. Discussion <a class="anchor" id="discussion"></a>

### 6.1 Successes <a class="anchor" id="success"></a>


## 7. Contributions. Who did what? <a class="anchor" id="contributions"></a>

**222798 - Laura**
- Clustering
- Data Preprocessing and EDA
- Correlation Matrix
- Writing motivation, genre & Discussion

**223724 - Julia**
- Created stacked barchart per city. 
- Created US geographical heatmap
- Found relevant research for references


**s223730 - Pierre**
- Found dataset, preprocessed data and performed EDA
- Formatted notebook and made website
- Made yearly Air Quality heatmap plot
- Data analysis
    - clustering based on transportation methods
    - found optimal number of clusters
    - eda based on each 3 clusters
    - made stacked bar chart based on clusters.
    - OLS based on clusters and comments for OLS results
- Helped writing discussion (limitations, successes)


## 8. References <a class="anchor" id="references"></a>

[1] Longhurst, J. W. S., D. Rayfield, and D. E. Conlan. 1994. "The Impacts Of Road Transport On Urban Air Quality-A Case Study Of The Greater Manchester Region." WIT Transactions on Ecology and the Environment 3.

[2] Jing, Qin-Lei, Han-Zhen Liu, Wei-Qing Yu, and Xu He. 2022. "The Impact of Public Transportation on Carbon Emissions—From the Perspective of Energy Consumption" Sustainability 14, no. 10: 6248. https://doi.org/10.3390/su14106248 

[3] Edward Segel and Jeffrey Heer. 2011 ."Narrative Visualization: Telling Stories with Data". IEEE transactions on visualization and computer graphics. 16. 1139-48. 